In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"



In [2]:
# All train
train = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test = ImageDataGenerator(rescale=1/255)

train_dataset_all = train.flow_from_directory("../input/fake-imagedetection/ALL train/ALL train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='training')

validation_dataset_all = train.flow_from_directory("../input/fake-imagedetection/ALL train/ALL train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='validation')                                    
test_dataset_all = test.flow_from_directory("../input/fake-imagedetection/All test/All test",
                                          target_size=(32,32),
                                          batch_size =32,
                                          class_mode = 'binary')

test_dataset_all.class_indices

In [3]:
# Bilinear
train = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test = ImageDataGenerator(rescale=1/255)

train_dataset_bl = train.flow_from_directory("../input/fake-imagedetection/RAML/bilinear/train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='training')

validation_dataset_bl = train.flow_from_directory("../input/fake-imagedetection/RAML/bilinear/train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='validation')                                    
test_dataset_bl = test.flow_from_directory("../input/fake-imagedetection/RAML/bilinear/test",
                                          target_size=(32,32),
                                          batch_size =32,
                                          class_mode = 'binary')

test_dataset_bl.class_indices

In [4]:
# Bicubic
train = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test = ImageDataGenerator(rescale=1/255)

train_dataset_bc = train.flow_from_directory("../input/fake-imagedetection/RAML/bicubic/train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='training')

validation_dataset_bc = train.flow_from_directory("../input/fake-imagedetection/RAML/bicubic/train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='validation')                                    
test_dataset_bc = test.flow_from_directory("../input/fake-imagedetection/RAML/bicubic/test",
                                          target_size=(32,32),
                                          batch_size =32,
                                          class_mode = 'binary')

test_dataset_bc.class_indices

In [5]:
# Pixxelshuffle
train = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test = ImageDataGenerator(rescale=1/255)

train_dataset_ps = train.flow_from_directory("../input/fake-imagedetection/RAML/pixelshuffle/train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='training')

validation_dataset_ps = train.flow_from_directory("../input/fake-imagedetection/RAML/pixelshuffle/train",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='validation')                                    
test_dataset_ps = test.flow_from_directory("../input/fake-imagedetection/RAML/pixelshuffle/test",
                                          target_size=(32,32),
                                          batch_size =32,
                                          class_mode = 'binary')

test_dataset_ps.class_indices

In [6]:
#All data= rotate + normal
train = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test = ImageDataGenerator(rescale=1/255)

train_dataset2 = train.flow_from_directory("../input/fake-imagedetection/All train RLall/All train R+L+all",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='training')

validation_dataset2 = train.flow_from_directory("../input/fake-imagedetection/All train RLall/All train R+L+all",
                                          target_size=(32,32),
                                          batch_size = 32,
                                          class_mode = 'binary',subset='validation')                                    
test_dataset2 = test.flow_from_directory("../input/fake-imagedetection/All test Rotate_LR/All test Rotate_L+R",
                                          target_size=(32,32),
                                          batch_size =32,
                                          class_mode = 'binary')

test_dataset2.class_indices

In [7]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)))
model.add(keras.layers.Dropout(0.3))
#model.add(keras.layers.MaxPool2D(2,2))

# # # Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.Dropout(0.3))
# model.add(keras.layers.MaxPool2D(2,2))

# # Convolutional layer and maxpool layer 3
#model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
#model.add(keras.layers.Dropout(0.3))
# model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(64,activation='relu'))

# Output layer with single neuron which gives 0 for fake or 1 for real 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))
opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
dataset_train=[train_dataset_all,train_dataset_bl,train_dataset_bc,train_dataset_ps, train_dataset2]
dataset_test=[test_dataset_all,test_dataset_bl,test_dataset_bc,test_dataset_ps,  test_dataset2]

In [9]:
# Train, validate and test the model for all the combination
early_stopping = EarlyStopping(
        min_delta=0.001, # minimium amount of change to count as an improvement
        patience=20, # how many epochs to wait before stopping
        restore_best_weights=True,
        )
for i in range(len(dataset_train)):
    history = model.fit(dataset_train[i],
        #  steps_per_epoch = 80,
             epochs = 150,
             callbacks=[early_stopping],
             validation_data = validation_dataset2
             )
    # visualizing losses and accuracy
    # print(history.history)
    train_loss = history.history['loss']
    val_loss   = history.history['val_loss']
    train_acc  = history.history['accuracy']
    val_acc    = history.history['val_accuracy']
    xc         = range(len(history.history['loss']))

    learning curves
    plt.figure()
    plt.plot(xc, train_loss, label = 'Train Loss')
    plt.plot(xc, val_loss, label = 'Val Loss')
    plt.figure()
    plt.plot(xc, train_acc,label = 'Train Acc')
    plt.plot(xc, val_acc,label = 'Val Acc')
    plt.legend(loc='upper right')
    
    if i == 0:
        _, acc_all = model.evaluate(dataset_test[i])
        
        
    if i == 1:
        _, acc_bl = model.evaluate(dataset_test[i])
        _, acc_bl_bc = model.evaluate(dataset_test[2])      
        _, acc_bl_ps = model.evaluate(dataset_test[3])
        
        
    if i == 2:
        _, acc_bc = model.evaluate(dataset_test[i])
        _, acc_bc_bl = model.evaluate(dataset_test[1]) 
        _, acc_bc_ps = model.evaluate(dataset_test[3])
        
    if i == 3:
        _, acc_ps = model.evaluate(dataset_test[i])
        _, acc_ps_bl = model.evaluate(dataset_test[1])
        _, acc_ps_bc = model.evaluate(dataset_test[2])
        
    if i==4:
        _, acc_rr = model.evaluate(dataset_test[i])



print()
print('Trained with All Three')
print('All: ' ,acc_all *100) 
print()

print('Trained with Bilinear')
print('Test with Bilinear: ' ,acc_bl *100)
print('Test with Bicubic: ' ,acc_bl_bc *100)
print('Test with PixelShuffle: ' ,acc_bl_ps *100)
print()

print('Trained with Bicubic')
print('Test with Bicubic: ' ,acc_bc *100)
print('Test with Bilinear: ' ,acc_bc_bl *100)
print('Test with PixelShuffle: ' ,acc_bc_ps *100)
print()

print('Trained with PixxelShuffle')
print('Test with PixxelShuffle: ' ,acc_ps *100)
print('Test with Bilinear: ' ,acc_ps_bl *100)
print('Test with Bicubic: ' ,acc_ps_bc *100)
print()
print('Trained with all rotate')
print('Test with all rotate:' , acc_rr *100)

In [10]:
def predictImage(filename):
    img1 = image.load_img(filename,target_size=(32,32))
    
    plt.imshow(img1)
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X)
    print(val)
    if val == 1:
        
        plt.xlabel("Real",fontsize=30)
        
    
    elif val == 0:
        
        plt.xlabel("Fake",fontsize=30)

predictImage(r"../input/fake-imagedetection/RAML/pixelshuffle/test/imagewoof_test/985.jpg")


In [11]:

predictImage(r"../input/fake-imagedetection/RAML/pixelshuffle/test/imagewoof_test/890.jpg")